In [7]:
import yfinance as yf
import pandas as pd

tickers = [
    "APOLLOHOSP.NS",
    "ADANIPOWER.NS",
    "DLF.NS",
    "GODREJCP.NS",
    "ADANIPORTS.NS",
    "EICHERMOT.NS",
    "TITAN.NS",
    "ONGC.NS",
    "BANKBARODA.NS",
    "TVSMOTOR.NS",
    "ADANIGREEN.NS",
    "TATAMOTORS.NS",
    "JSWSTEEL.NS",
    "TORNTPHARM.NS",
    "HINDZINC.NS",
    "TECHM.NS",
    "PFC.NS",
    "VEDL.NS",
    "ICICIBANK.NS",
    "GAIL.NS",
    "INFY.NS",
    "MCDOWELL-N.NS",
    "ADANIGREEN.NS",
    "PIDILITIND.NS",
    "BAJAJHLDNG.NS",
    "SBIN.NS",
    "COALINDIA.NS",
]

financials_dict = {}
for ticker in tickers:
    stock = yf.Ticker(ticker)

    # Fetching income statement data
    income_statement = stock.financials
    income_statement.dropna(axis=0, how="all", inplace=True)
    income_statement = income_statement.iloc[:, :3]
    financials_dict[ticker + "_income_statement"] = income_statement

    # Fetching balance sheet data
    balance_sheet = stock.balance_sheet
    balance_sheet.dropna(axis=0, how="all", inplace=True)
    balance_sheet = balance_sheet.iloc[:, :3]
    financials_dict[ticker + "_balance_sheet"] = balance_sheet

    # Fetching cash flow data
    cash_flow = stock.cashflow
    cash_flow.dropna(axis=0, how="all", inplace=True)
    cash_flow = cash_flow.iloc[:, :3]
    financials_dict[ticker + "_cash_flow"] = cash_flow

    toSave_csv = pd.DataFrame(columns=cash_flow.columns)
    toSave_csv = pd.concat([income_statement, balance_sheet, cash_flow])
    toSave_csv.to_csv(f"data/{ticker}.csv")

In [13]:
# selecting relevant financial information for each stock using fundamental data
stats = [
    # "Net income applicable to common shares",
    "Total Assets",
    # "Total cash flow from operating activities",
    # "Long-term debt",
    # "Other liabilities",
    # "Total current assets",
    # "Total current liabilities",
    # "Common stock",
    # "Total revenue",
    # "Gross profit",
]  # change as required

indx = [
    "NetIncome",
    "TotAssets",
    "CashFlowOps",
    "LTDebt",
    "OtherLTDebt",
    "CurrAssets",
    "CurrLiab",
    "CommStock",
    "TotRevenue",
    "GrossProfit",
]

In [14]:
def info_filter(df, stats, indx):
    """function to filter relevant financial information for each
    stock and transforming string inputs to numeric"""
    tickers = df.columns
    all_stats = {}
    for ticker in tickers:
        try:
            temp: pd.Series = df[ticker]
            ticker_stats = []
            for stat in stats:
                ticker_stats.append(temp.loc[stat])
            all_stats["{}".format(ticker)] = ticker_stats
        except Exception as e:
            print("can't read data for ", ticker)

    all_stats_df = pd.DataFrame(all_stats, index=indx)

    # cleansing of fundamental data imported in dataframe
    all_stats_df[tickers] = all_stats_df[tickers].replace({",": ""}, regex=True)
    for ticker in all_stats_df.columns:
        all_stats_df[ticker] = pd.to_numeric(
            all_stats_df[ticker].values, errors="coerce"
        )
    return all_stats_df

In [15]:
def piotroski_f(df_cy, df_py, df_py2):
    """function to calculate f score of each stock and output information as dataframe"""
    f_score = {}
    tickers = df_cy.columns
    for ticker in tickers:
        ROA_FS = int(
            df_cy.loc["NetIncome"].to_list()[0]
            / (
                (
                    df_cy.loc["TotAssets"].to_list()[0]
                    + df_py.loc["TotAssets"].to_list()[0]
                )
                / 2
            )
            > 0
        )
        CFO_FS = int(df_cy.loc["CashFlowOps"].to_list()[0] > 0)
        ROA_D_FS = int(
            df_cy.loc["NetIncome"].to_list()[0]
            / (
                df_cy.loc["TotAssets"].to_list()[0]
                + df_py.loc["TotAssets"].to_list()[0]
            )
            / 2
            > df_py.loc["NetIncome"].to_list()[0]
            / (
                df_py.loc["TotAssets"].to_list()[0]
                + df_py2.loc["TotAssets"].to_list()[0]
            )
            / 2
        )
        CFO_ROA_FS = int(
            df_cy.loc["CashFlowOps"].to_list()[0] / df_cy.loc["TotAssets"].to_list()[0]
            > df_cy.loc["NetIncome"].to_list()[0]
            / (
                (
                    df_cy.loc["TotAssets"].to_list()[0]
                    + df_py.loc["TotAssets"].to_list()[0]
                )
                / 2
            )
        )
        LTD_FS = int(
            (df_cy.loc["LTDebt"].to_list()[0] + df_cy.loc["OtherLTDebt"].to_list()[0])
            < (df_py.loc["LTDebt"].to_list()[0] + df_py.loc["OtherLTDebt"].to_list()[0])
        )
        CR_FS = int(
            (df_cy.loc["CurrAssets"].to_list()[0] / df_cy.loc["CurrLiab"].to_list()[0])
            > (
                df_py.loc["CurrAssets"].to_list()[0]
                / df_py.loc["CurrLiab"].to_list()[0]
            )
        )
        DILUTION_FS = int(
            df_cy.loc["CommStock"].to_list()[0] <= df_py.loc["CommStock"].to_list()[0]
        )
        GM_FS = int(
            (
                df_cy.loc["GrossProfit"].to_list()[0]
                / df_cy.loc["TotRevenue"].to_list()[0]
            )
            > (
                df_py.loc["GrossProfit"].to_list()[0]
                / df_py.loc["TotRevenue"].to_list()[0]
            )
        )
        ATO_FS = int(
            df_cy.loc["TotRevenue"].to_list()[0]
            / (
                (
                    df_cy.loc["TotAssets"].to_list()[0]
                    + df_py.loc["TotAssets"].to_list()[0]
                )
                / 2
            )
            > df_py.loc["TotRevenue"].to_list()[0]
            / (
                (
                    df_py.loc["TotAssets"].to_list()[0]
                    + df_py2.loc["TotAssets"].to_list()[0]
                )
                / 2
            )
        )
        f_score[ticker] = [
            ROA_FS,
            CFO_FS,
            ROA_D_FS,
            CFO_ROA_FS,
            LTD_FS,
            CR_FS,
            DILUTION_FS,
            GM_FS,
            ATO_FS,
        ]
    f_score_df = pd.DataFrame(
        f_score,
        index=[
            "PosROA",
            "PosCFO",
            "ROAChange",
            "Accruals",
            "Leverage",
            "Liquidity",
            "Dilution",
            "GM",
            "ATO",
        ],
    )
    return f_score_df.sum().values[0]

In [16]:
import glob

fp = glob.glob("data/*.csv")

In [20]:
# Apply info_filter function on income statement data
import glob

tickers = []
p_scores = []
fp = glob.glob("data/*.csv")
for file_path in fp:
    ticker_df = pd.read_csv(file_path, index_col=0)
    filtered_data = info_filter(ticker_df, stats, indx)
    val = piotroski_f(
        pd.DataFrame(
            filtered_data["2023-03-31"].to_list(),
            columns=["2023-03-31"],
            index=filtered_data.index.to_list(),
        ),
        pd.DataFrame(
            filtered_data["2022-03-31"].to_list(),
            columns=["2022-03-31"],
            index=filtered_data.index.to_list(),
        ),
        pd.DataFrame(
            filtered_data["2021-03-31"].to_list(),
            columns=["2021-03-31"],
            index=filtered_data.index.to_list(),
        ),
    )
    tickers.append(file_path[5:-4])
    p_scores.append(val)

f_scores_df = pd.DataFrame({"Tickers": tickers, "piotroski_score": p_scores})
f_scores_df.sort_values(by="piotroski_score", ascending=False, inplace=True)
f_scores_df.to_csv("results/f_scores.csv")
f_scores_df

,Tickers,piotroski_score
13,INFY.NS,5
4,BAJAJHLDNG.NS,5
20,TATAMOTORS.NS,5
11,HINDZINC.NS,5
25,VEDL.NS,5
5,BANKBARODA.NS,4
6,COALINDIA.NS,4
24,TVSMOTOR.NS,4
23,TORNTPHARM.NS,4
2,ADANIPOWER.NS,4
